In [62]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
from scipy import stats, integrate

In [63]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOW_SpeedLimit.csv', encoding='utf-8')
herepath.head()

,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,SPDLIMIT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT_5,PCT_50,PCT_85,DOW
0,998969850F,2014-01-01 13:55,167,186,72,61.0,0.0,61,61,10,61,61,61,Wednesday
1,998969850F,2014-01-02 11:05,133,186,72,60.0,0.0,60,60,10,60,60,60,Thursday
2,998969850F,2014-01-02 13:20,160,186,72,61.0,0.0,61,61,10,61,61,61,Thursday
3,998969850F,2014-01-02 14:30,174,186,72,52.0,0.0,52,52,10,52,52,52,Thursday
4,998969850F,2014-01-03 09:05,109,186,72,57.0,0.0,57,57,10,57,57,57,Friday


In [64]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [65]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [66]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [67]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe(w,x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        link = v[v['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        link = x[x['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [68]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [69]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsBase(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y = pd.DataFrame(columns=[])
    
    y['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thAll'] = epoct
    y['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
   
    
    y['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    y['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
      

    
    y = np.round(y,3)
    return z

In [70]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsALL(x):
    y = pd.DataFrame(columns=[])
    
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thAll'] = epoct
    y['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
    y = np.round(y,3)
    return y

In [71]:
def IndecesALLAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_AM'] = y['SPDLIMIT'].div(y['MEAN'])
    z['PTI_SL_AM'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85thALL_AM'] = y['PcntSpeed_85thAll'].div(y['MEAN'])
    z['PTI_85thALL_AM'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_AM'] = (y['MEAN'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [72]:
def IndecesALLMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_MD'] = y['SPDLIMIT'].div(y['MEAN'])
    z['PTI_SL_MD'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85th_MD'] = y['PcntSpeed_85thAll'].div(y['MEAN'])
    z['PTI_85th_MD'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_MD'] = (y['MEAN'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [73]:
def IndecesALLPM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_PM'] = y['SPDLIMIT'].div(y['MEAN'])
    z['PTI_SL_PM'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85th_PM'] = y['PcntSpeed_85thAll'].div(y['MEAN'])
    z['PTI_85th_PM'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_PM'] = (y['MEAN'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [74]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWd(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)

    
    y['PcntSpeed_85thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWd'] = epoct
    y['PcntInterval_85thWd'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWd'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [75]:
def IndecesWdAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_AM'] = y['PcntSpeed_85thWd'].div(y['MEAN'])
    z['PTI_85thWd_AM'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [76]:
def IndecesWdMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_MD'] = y['PcntSpeed_85thWd'].div(y['MEAN'])
    z['PTI_85thWd_MD'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [77]:
def IndecesWdPM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_PM'] = y['PcntSpeed_85thWd'].div(y['MEAN'])
    z['PTI_85thWd_PM'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [78]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWend(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thWend'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWend'] = epoct
    y['PcntInterval_85thWend'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWend'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [79]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdtime(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_60thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_85thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_Wdtime'] = epoct
    y['PcntInterval_Wdtime'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_Wdtime'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [80]:
def IndecesWDtimeAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_AM'] = y['PcntSpeed_60thWdtime'].div(y['MEAN'])
    z['PTI_60thWdtime_AM'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_AM'] = y['PcntSpeed_85thWdtime'].div(y['MEAN'])
    z['PTI_85thWdtime_AM'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [81]:
def IndecesWDtimeMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_MD'] = y['PcntSpeed_60thWdtime'].div(y['MEAN'])
    z['PTI_60thWdtime_MD'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_MD'] = y['PcntSpeed_85thWdtime'].div(y['MEAN'])
    z['PTI_85thWdtime_MD'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [82]:
def IndecesWDtimePM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_PM'] = y['PcntSpeed_60thWdtime'].div(y['MEAN'])
    z['PTI_60thWdtime_PM'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_PM'] = y['PcntSpeed_85thWdtime'].div(y['MEAN'])
    z['PTI_85thWdtime_PM'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [83]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayAM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayAM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayAM'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayAM'] = epoct
    y['PcntInterval_WdayAM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayAM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [84]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayMD(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayMD'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayMD'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayMD'] = epoct
    y['PcntInterval_WdayMD'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayMD'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [85]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayPM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayPM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayPM'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayPM'] = epoct
    y['PcntInterval_WdayPM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayPM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [86]:
#For all days and all time periods
All = timeframe2(herepath,0,288,False,Both)
bylink = HEREstatsALL(All)
bylink = bylink.drop(['MEAN','SPDLIMIT'], axis = 1)
All = pd.DataFrame()
bylink.head()

,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll
LINK_DIR,,,,,
107030432T,48.461,19.260,5746,5.466,10.444
107030439T,44.734,17.396,7857,7.474,9.929
107030440T,45.355,17.086,7325,6.968,10.303
107030444F,47.219,16.154,9311,8.857,11.209
107030445F,47.840,16.775,9256,8.805,11.250


In [87]:
#for weekdays all links
Wkd = timeframe2(herepath,0,288,True,Weekday)
SpeedWd = HEREstatsWd(Wkd)
SpeedWd = SpeedWd.drop(['MEAN','SPDLIMIT'], axis = 1)
Wkd = pd.DataFrame()
SpeedWd.head()

,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,,
107030432T,47.840,19.260,5023,6.682,10.344
107030439T,44.734,17.396,6775,9.013,9.937
107030440T,45.355,17.396,6339,8.433,10.262
107030444F,47.219,15.532,8045,10.703,11.301
107030445F,47.840,16.775,8008,10.653,11.282


In [88]:
#for weekends all links
Wend = timeframe2(herepath,0,288,True,Weekend)
SpeedWend = HEREstatsWend(Wend)
SpeedWend = SpeedWend.drop(['MEAN','SPDLIMIT'], axis = 1)
Wend = pd.DataFrame()
SpeedWend.head()

,PcntSpeed_85thWend,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend
LINK_DIR,,,,
107030432T,50.325,723,2.414,10.988
107030439T,45.355,1082,3.612,9.874
107030440T,45.355,986,3.292,10.532
107030444F,47.840,1266,4.227,10.461
107030445F,49.083,1248,4.167,10.760


In [89]:
#Weekday Between 6AM and 8PM all links
Wdtime = timeframe2(herepath,72,240,True,Weekday)
SpeedWdtime = HEREstatsWdtime(Wdtime)
SpeedWdtime = SpeedWdtime.drop(['MEAN','SPDLIMIT'], axis = 1)
Wdtime = pd.DataFrame()
SpeedWdtime.head()

,PcntSpeed_60thWdtime,PcntSpeed_85thWdtime,PcntSpeed_5thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime
LINK_DIR,,,,,,
107030432T,41.627,47.840,18.639,4458,5.931,10.412
107030439T,38.521,44.734,17.396,6233,8.292,9.992
107030440T,37.899,45.355,16.775,5891,7.837,10.269
107030444F,40.198,47.219,14.911,7538,10.028,11.354
107030445F,40.384,47.219,16.154,7355,9.785,11.290


In [90]:
#Stats between 6AM and 9AM all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
SpeedWdayAM = HEREstatsWdayAM(WdtimeAM)
SpeedWdayAM = SpeedWdayAM.drop(['MEAN','SPDLIMIT'], axis = 1)
WdtimeAM = pd.DataFrame()
SpeedWdayAM.head()

,PcntSpeed_5thWdayAM,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM
LINK_DIR,,,,,
107030432T,19.260,38.661,735,0.978,10.649
107030439T,16.775,34.048,991,1.318,10.676
107030440T,16.154,34.625,852,1.133,11.012
107030444F,14.911,33.387,1313,1.747,11.931
107030445F,15.532,33.241,1340,1.783,11.442


In [91]:
#Stats between 9AM and 3PM all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
SpeedWdayMD = HEREstatsWdayMD(WdtimeMD)
SpeedWdayMD = SpeedWdayMD.drop(['MEAN','SPDLIMIT'], axis = 1)
WdtimeMD = pd.DataFrame()
SpeedWdayMD.head()

,PcntSpeed_5thWdayMD,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD
LINK_DIR,,,,,
107030432T,18.639,38.016,2168,2.884,10.330
107030439T,18.639,35.201,3283,4.368,9.513
107030440T,17.396,34.843,3159,4.203,9.932
107030444F,16.775,37.005,3910,5.202,10.765
107030445F,17.396,37.109,3840,5.109,10.932


In [92]:
#Stats between 3PM and 6PM all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
SpeedWdayPM = HEREstatsWdayPM(WdtimePM)
SpeedWdayPM = SpeedWdayPM.drop(['MEAN','SPDLIMIT'], axis = 1)
WdtimePM = pd.DataFrame()
SpeedWdayPM.head()

,PcntSpeed_5thWdayPM,AvgSpeed_WdayPM,TotalIntervals_WdayPM,PcntInterval_WdayPM,StdDev_WdayPM
LINK_DIR,,,,,
107030432T,18.639,36.940,1285,1.710,10.080
107030439T,15.532,34.572,1527,2.031,10.245
107030440T,16.154,34.442,1506,2.004,10.287
107030444F,11.805,33.683,1821,2.423,11.693
107030445F,14.290,34.508,1750,2.328,11.566


In [93]:
#Calculates AM Indeces for all 365 days for all links
AllAM = timeframe2(herepath,72,108,False,Both)
HstatsAM = HEREstatsALL(AllAM)
IndecesAllAM = IndecesALLAM(HstatsAM)
AllAM = pd.DataFrame()
IndecesAllAM.head()

,TTI_SL_AM,PTI_SL_AM,TTI_85thALL_AM,PTI_85thALL_AM,BTI_AM
LINK_DIR,,,,,
107030432T,1.276,2.581,1.244,2.516,1.023
107030439T,1.309,2.667,1.327,2.704,1.037
107030440T,1.293,2.769,1.329,2.846,1.142
107030444F,1.325,3.000,1.398,3.167,1.265
107030445F,1.331,2.880,1.387,3.000,1.163


In [94]:
#Calculates MD Indeces for all 365 days for all links
AllMD = timeframe2(herepath,108,180,False,Both)
HstatsMD = HEREstatsALL(AllMD)
IndecesAllMD = IndecesALLMD(HstatsMD)
AllMD = pd.DataFrame()
IndecesAllMD.head()

,TTI_SL_MD,PTI_SL_MD,TTI_85th_MD,PTI_85th_MD,BTI_MD
LINK_DIR,,,,,
107030432T,1.295,2.581,1.247,2.484,0.992
107030439T,1.272,2.400,1.272,2.400,0.887
107030440T,1.293,2.572,1.293,2.572,0.989
107030444F,1.209,2.618,1.285,2.782,1.165
107030445F,1.200,2.483,1.284,2.655,1.069


In [95]:
#Calculates PM Indeces for all 365 days for all links
AllPM = timeframe2(herepath,180,216,False,Both)
HstatsPM = HEREstatsALL(AllPM)
IndecesAllPM = IndecesALLPM(HstatsPM)
AllPM = pd.DataFrame()
IndecesAllPM.head()

,TTI_SL_PM,PTI_SL_PM,TTI_85th_PM,PTI_85th_PM,BTI_PM
LINK_DIR,,,,,
107030432T,1.334,2.667,1.267,2.533,0.999
107030439T,1.291,2.880,1.291,2.880,1.231
107030440T,1.302,2.880,1.302,2.880,1.212
107030444F,1.307,3.600,1.343,3.700,1.754
107030445F,1.272,3.000,1.343,3.167,1.358


In [96]:
#Calculates weekday non-holiday indeces for AM for all links
WdAM = timeframe2(herepath,72,108,True,Weekday)
HstatsWdAM = HEREstatsWd(WdAM)
IndecesWdayAM = IndecesWdAM(HstatsWdAM)
WdAM = pd.DataFrame()
IndecesWdayAM.head()

,TTI_85thWd_AM,PTI_85thWd_AM
LINK_DIR,,
107030432T,1.253,2.516
107030439T,1.332,2.704
107030440T,1.328,2.846
107030444F,1.396,3.125
107030445F,1.383,2.960


In [ ]:
#Calculates weekday non-holiday indeces for MD for all links
WdMD = timeframe2(herepath,108,180,True,Weekday)
HstatsWdMD = HEREstatsWd(WdMD)
IndecesWdayMD = IndecesWdMD(HstatsWdMD)
WdMD = pd.DataFrame()
IndecesWdayMD.head()

,TTI_85thWd_MD,PTI_85thWd_MD
LINK_DIR,,
107030432T,1.258,2.567
107030439T,1.271,2.400
107030440T,1.284,2.572
107030444F,1.293,2.852
107030445F,1.289,2.750


In [ ]:
#Calculates weekday non-holiday indeces for PM for all links
WdPM = timeframe2(herepath,180,216,True,Weekday)
HstatsWdPM = HEREstatsWd(WdPM)
IndecesWdayPM = IndecesWdPM(HstatsWdPM)
WdPM = pd.DataFrame()
IndecesWdayPM.head()

In [ ]:
#Calculates daytime indeces for AM for all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
HstatsWdtimeAM = HEREstatsWdtime(WdtimeAM)
IndecesWdtimeAM = IndecesWDtimeAM(HstatsWdtimeAM)
WdtimeAM = pd.DataFrame()
IndecesWdtimeAM.head()

In [ ]:
#Calculates daytime indeces for MD for all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
HstatsWdtimeMD = HEREstatsWdtime(WdtimeMD)
IndecesWdtimeMD = IndecesWDtimeMD(HstatsWdtimeMD)
WdtimeMD = pd.DataFrame()
IndecesWdtimeMD.head()

In [ ]:
#Calculates daytime indeces for PM for all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
HstatsWdtimePM = HEREstatsWdtime(WdtimePM)
IndecesWdtimePM = IndecesWDtimePM(HstatsWdtimePM)
WdtimePM = pd.DataFrame()
IndecesWdtimePM.head()

In [ ]:
master = bylink

In [ ]:
master = pd.merge(master, SpeedWd, left_index = True, right_index = True)
SpeedWd = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWend, left_index = True, right_index = True)
SpeedWend = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWdtime, left_index = True, right_index = True)
SpeedWdtime = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWdayAM, left_index = True, right_index = True)
SpeedWdayAM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWdayMD, left_index = True, right_index = True)
SpeedWdayMD = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, SpeedWdayPM, left_index = True, right_index = True)
SpeedWdayPM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesAllAM, left_index = True, right_index = True)
IndecesAllAM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesAllMD, left_index = True, right_index = True)
IndecesAllMD = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesAllPM, left_index = True, right_index = True)
IndecesAllPM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdayAM, left_index = True, right_index = True)
IndecesWdayAM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdayMD, left_index = True, right_index = True)
IndecesWdayMD = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdayPM, left_index = True, right_index = True)
IndecesWdayPM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdtimeAM, left_index = True, right_index = True)
IndecesWdtimeAM = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdtimeMD, left_index = True, right_index = True)
IndecesWdtimeMD = pd.DataFrame()
master.head()

In [ ]:
master = pd.merge(master, IndecesWdtimePM, left_index = True, right_index = True)
IndecesWdtimePM = pd.DataFrame()
master.head()

In [ ]:
master.to_csv('HEREstats.csv')